In [12]:
!wget http://www.kasprowski.pl/datasets/emvic.zip
!unzip emvic.zip


--2024-10-20 13:33:14--  http://www.kasprowski.pl/datasets/emvic.zip
Resolving www.kasprowski.pl (www.kasprowski.pl)... 94.152.164.213
Connecting to www.kasprowski.pl (www.kasprowski.pl)|94.152.164.213|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4295994 (4.1M) [application/zip]
Saving to: ‘emvic.zip’

emvic.zip           100%[===================>]   4.10M  3.50MB/s    in 1.2s    

2024-10-20 13:33:16 (3.50 MB/s) - ‘emvic.zip’ saved [4295994/4295994]

Archive:  emvic.zip
  inflating: emvic.data              


In [21]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import GradientBoostingClassifier, BaggingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import cross_val_score
import joblib
from collections import Counter

In [14]:
# Wczytanie danych
data = pd.read_csv('emvic.data', sep='\t', header=None)

print(data.head())


  0      1      2      3      4      5      6      7      8      9      ...  \
0   a25      0      0      0      0      0      0      0      0      0  ...   
1   a25      0      0      0      0      0      0      0      0      0  ...   
2   a25      0      0      0      0      0      0      0      0      0  ...   
3   a25      0      0      0      0      0      0      0      0      0  ...   
4   a25      0      0      0      0      0      0      0      0      0  ...   

   12279  12280  12281  12282  12283  12284  12285  12286  12287  12288  
0    -20    -20    -20    -20    -19    -20    -22    -19    -22    -23  
1    -74    -73    -75    -73    -72    -72    -69    -67    -67    -70  
2    -51    -52    -54    -51    -54    -54    -54    -53    -52    -54  
3     74     74     74     74     74     74     75     71     76     74  
4    129    130    130    133    131    129    124    124    122    117  

[5 rows x 12289 columns]


In [15]:

# Podział danych na klasy i cechy
classes = data.iloc[:, 0] #etykiety klas
lx = data.iloc[:, 4097:6145]  #ruch lewego oka w osi X
rx = data.iloc[:, 6145:8193] #ruch prawego oka w osi X
ly = data.iloc[:, 8193:10241] #ruch lewego oka w osi Y
ry = data.iloc[:, 10241:] #ruch prawego oka w osi Y
displayed_point_x = data.iloc[:, 1:2049] #Kolumny 1 do 2049: pozycja X wyświetlanego punktu
displayed_point_y = data.iloc[:, 2049:4097] #Kolumny 2049 do 4097: pozycja Y wyświetlanego punktu

print("Liczba próbek: ", data.shape[0])
print("Wymiary danych dla punktu wyświetlanego (X, Y): ", displayed_point_x.shape, displayed_point_y.shape)
print("Wymiary danych dla ruchu oczu (Lewe oko X, Y): ", lx.shape, ly.shape)
print("Wymiary danych dla ruchu oczu (Prawe oko X, Y): ", rx.shape, ry.shape)

samples = np.hstack([lx.values, rx.values, ly.values, ry.values])
labels = classes.values

# Podział danych na treningowe i testowe
(trainSamples, testSamples, trainLabels, testLabels) = train_test_split(samples, labels, random_state=30, stratify=labels)
print("Rozkład etykiet w zbiorze treningowym:", Counter(trainLabels))
print("Rozkład etykiet w zbiorze testowym:", Counter(testLabels))




Liczba próbek:  416
Wymiary danych dla punktu wyświetlanego (X, Y):  (416, 2048) (416, 2048)
Wymiary danych dla ruchu oczu (Lewe oko X, Y):  (416, 2048) (416, 2048)
Wymiary danych dla ruchu oczu (Prawe oko X, Y):  (416, 2048) (416, 2048)
Rozkład etykiet w zbiorze treningowym: Counter({'a25': 79, 'a37': 47, 'a40': 39, 'a32': 38, 'a41': 29, 'a42': 28, 'a29': 27, 'a28': 25})
Rozkład etykiet w zbiorze testowym: Counter({'a25': 26, 'a37': 16, 'a40': 13, 'a32': 12, 'a41': 10, 'a42': 10, 'a29': 9, 'a28': 8})


In [16]:

#Podział danych na zbiory testowe i treningowe
trainSamples, testSamples, trainLabels, testLabels = train_test_split(
    samples, labels, test_size=0.3, random_state=42, stratify=labels)

# Normalizacja danych
scaler = StandardScaler()
trainSamples = scaler.fit_transform(trainSamples)
testSamples = scaler.transform(testSamples)

In [17]:
#funkcja do oceny klasyfikatorów
def evaluate_classifier(clf, name):
    clf.fit(trainSamples, trainLabels)
    predictions = clf.predict(testSamples)
    accuracy = accuracy_score(testLabels, predictions)
    precision = precision_score(testLabels, predictions, average='weighted')
    recall = recall_score(testLabels, predictions, average='weighted')
    f1 = f1_score(testLabels, predictions, average='weighted')
    print(f"\n{name}:")
    print(f"Dokładność: {accuracy:.2f}, Precyzja: {precision:.2f}, Czułość: {recall:.2f}, F1: {f1:.2f}")
    print(confusion_matrix(testLabels, predictions))


In [ ]:
#Wykorzystanie klasyfikatora Random Forest:
clf_rf = RandomForestClassifier(n_estimators=100, class_weight='balanced')
evaluate_classifier(clf_rf, "Random Forest")


Random Forest:
Dokładność: 0.78, Precyzja: 0.78, Czułość: 0.78, F1: 0.77
[[29  0  0  1  0  0  0  1]
 [ 0  5  1  0  3  0  1  0]
 [ 0  2  9  0  0  0  0  0]
 [ 5  0  1  8  1  0  0  0]
 [ 1  0  0  0 15  0  1  2]
 [ 0  0  0  0  0 16  0  0]
 [ 0  2  0  0  2  0  7  1]
 [ 0  0  0  1  1  1  0  8]]


In [ ]:
#Wykorzystanie klasyfikatora Gradient Boosting:
clf_gb = GradientBoostingClassifier(n_estimators=100)
evaluate_classifier(clf_gb, "Gradient Boosting")


Gradient Boosting:
Dokładność: 0.74, Precyzja: 0.73, Czułość: 0.74, F1: 0.73
[[29  0  0  0  0  0  1  1]
 [ 0  5  1  0  2  1  0  1]
 [ 0  1  9  0  1  0  0  0]
 [ 4  0  2  7  2  0  0  0]
 [ 0  2  1  1 13  0  1  1]
 [ 0  0  0  1  0 15  0  0]
 [ 0  1  1  0  2  1  6  1]
 [ 0  0  0  2  0  1  0  8]]


In [ ]:
#Wykorzystanie klasyfikatora KNN:
clf_knn = KNeighborsClassifier(n_neighbors=5)
evaluate_classifier(clf_knn, "KNN")


KNN:
Dokładność: 0.64, Precyzja: 0.67, Czułość: 0.64, F1: 0.63
[[29  0  0  2  0  0  0  0]
 [ 0  6  1  0  3  0  0  0]
 [ 0  1  9  1  0  0  0  0]
 [ 7  0  1  6  0  0  0  1]
 [ 6  1  2  0  7  1  0  2]
 [ 3  2  0  0  0 11  0  0]
 [ 2  0  2  0  3  0  5  0]
 [ 1  1  1  1  0  0  0  7]]


In [ ]:
#Wykorzystanie klasyfikatora SVM:
clf_svm = SVC(kernel='linear', C=1)
evaluate_classifier(clf_svm, "SVM")


SVM:
Dokładność: 0.90, Precyzja: 0.92, Czułość: 0.90, F1: 0.91
[[29  0  0  0  0  0  0  2]
 [ 0  8  0  0  0  0  0  2]
 [ 0  1 10  0  0  0  0  0]
 [ 0  0  2 11  1  0  0  1]
 [ 0  1  0  0 17  0  0  1]
 [ 0  0  0  0  0 16  0  0]
 [ 0  0  0  0  1  0 11  0]
 [ 0  0  0  0  0  0  0 11]]


In [ ]:
#Wykorzystanie klasyfikatora Logistic Regression:
clf_lr = LogisticRegression(C=1, solver='lbfgs', max_iter=1000)
evaluate_classifier(clf_lr, "Logistic Regression")


Logistic Regression:
Dokładność: 0.92, Precyzja: 0.93, Czułość: 0.92, F1: 0.92
[[29  0  0  1  0  0  0  1]
 [ 0  8  0  0  0  0  0  2]
 [ 0  1 10  0  0  0  0  0]
 [ 0  0  0 14  1  0  0  0]
 [ 0  0  0  0 16  0  1  2]
 [ 0  0  0  0  0 16  0  0]
 [ 0  0  0  0  0  0 12  0]
 [ 0  1  0  0  0  0  0 10]]


In [ ]:
#Wykorzystanie klasyfikatora Decision Tree:
clf_tree = DecisionTreeClassifier(max_depth=100)
evaluate_classifier(clf_tree, "Decision Tree")


Decision Tree:
Dokładność: 0.58, Precyzja: 0.58, Czułość: 0.58, F1: 0.56
[[27  0  0  1  0  0  2  1]
 [ 0  4  1  0  0  1  1  3]
 [ 0  0 10  1  0  0  0  0]
 [ 4  1  1  5  1  0  1  2]
 [ 1  2  1  2  5  2  0  6]
 [ 2  0  0  0  0 13  1  0]
 [ 0  3  0  0  1  1  3  4]
 [ 1  1  0  1  3  0  0  5]]


In [18]:
# Wykorzystanie klasyfikatora Ridge Classifier:
ridge_classifier = RidgeClassifier()
evaluate_classifier(ridge_classifier, "Ridge Classifier")


Ridge Classifier:
Dokładność: 0.87, Precyzja: 0.88, Czułość: 0.87, F1: 0.87
[[28  0  0  2  0  0  0  1]
 [ 0  8  0  0  0  0  1  1]
 [ 0  0 10  0  0  1  0  0]
 [ 0  0  2 13  0  0  0  0]
 [ 0  0  0  0 17  0  1  1]
 [ 0  0  0  0  2 14  0  0]
 [ 1  0  0  1  0  0 10  0]
 [ 0  2  0  0  0  0  0  9]]


In [20]:
# cross walidacja:

samples = np.hstack([lx.values, rx.values, ly.values, ry.values])
labels = classes.values

classifiers_cross_val = {
    'SVM': SVC(kernel='rbf', C=1),
    'Ridge Classifier': RidgeClassifier(),
    'Logistic Regression': LogisticRegression(C=1, solver='lbfgs'),
}

results = {}
for name, clf in classifiers_cross_val.items():
    scores = cross_val_score(clf, samples, labels, cv=5)
    results[name] = scores
    print(f"{name}: Accuracy: {np.mean(scores):.4f} ± {np.std(scores):.4f}")

SVM: Accuracy: 0.7909 ± 0.0529
Ridge Classifier: Accuracy: 0.8606 ± 0.0319
Logistic Regression: Accuracy: 0.8773 ± 0.0329


In [24]:
# Zastosowanie SelectKBest do redukcji wymiarów
selector = SelectKBest(f_classif, k=1500)
trainSamples_selected = selector.fit_transform(trainSamples, trainLabels)
testSamples_selected = selector.transform(testSamples)

# Wybrany klasyfikator - Logistic Regression po selekcji cech
clf_lr_selected = LogisticRegression(C=1, solver='lbfgs', max_iter=1000)
clf_lr_selected.fit(trainSamples_selected, trainLabels)
predictions_lr_selected = clf_lr_selected.predict(testSamples_selected)
accuracy_lr_selected = accuracy_score(testLabels, predictions_lr_selected)
print(f"Dokładność Logistic Regression po selekcji cech: {accuracy_lr_selected:.2f}")


Dokładność Logistic Regression po selekcji cech: 0.90


In [26]:
# Zapis modelu
joblib.dump(clf_lr_selected, 'logistic_reg.joblib')

['logistic_reg.joblib']